# A Vector Space Model, with scikit-learn

In [ ]:
# This is code to build a vector space model, with SVMs on Andrew Mass' 
# distribution of movie review sentiment data.

In [2]:
from collections import namedtuple

all_data = []  
DataDoc= namedtuple('DataDoc', 'tag words')
with open('/Users/mam/CORE/RESEARCH/DEEPLEARNING/Doc2Vec/data/aclImdb/alldata-id.txt') as alldata:
    for line_no, line in enumerate(alldata):
        label=line.split()[0]
        word_list=line.lower().split()[1:]
        all_data.append(DataDoc(label, word_list))
        #print my_data[line_no]
        #break
train_data = all_data[:25000]
test_data = all_data[25000:50000]
print len(train_data)

#train_data=train_data[:100]+train_data[12500:12600]
#test_data=test_data[:100]+test_data[12500:12600]
print len(train_data)
print len(test_data)


25000
25000
25000


In [3]:
# Let's get a dictionary of all the words in training data
# These will be our bag-of-words features
# We won't need this function, since we will use gensim's built-in method "Dictionary" from the corpus module
# --> corpora.Dictionary, but we provide this so that you are clear on one way of how to do this.
from collections import defaultdict
def get_space(train_data):
    """
    input is a list of namedtuples
    get a dict of word space
    key=word
    value=len of the dict at that point 
    (that will be the index of the word and it is unique since the dict grows as we loop)
    """
    word_space=defaultdict(int)
    for doc in train_data:
        for w in doc.words:
            # indexes of words won't be in sequential order as they occur in data (can you tell why?), 
            # but that doesn't matter.
            word_space[w]=len(word_space)
    return word_space

word_space=get_space(train_data)
print len(word_space)
print word_space["love"]

113562
113538


In [4]:
import numpy as np

def get_sparse_vec(data_point, space):
    # create empty vector
    sparse_vec = np.zeros((len(space)))
    for w in set(data_point.words):
        # use exception handling such that this function can also be used to vectorize 
        # data with words not in train (i.e., test and dev data)
        try:
            sparse_vec[space[w]]=1
        except:
            continue
    return sparse_vec

 

train_vecs= [get_sparse_vec(data_point, word_space) for data_point in train_data]
test_vecs= [get_sparse_vec(data_point, word_space) for data_point in test_data]
#test_vecs= get_sparse_vectors(test_data, word_space)

#print train_vecs, test_vecs[0]
print len(train_data)
print len(train_vecs)
print len(test_vecs)

25000
25000
25000


In [4]:
# We should usually get tags automatically based on input data file.
# In the input data file we have, we know that the first 12500 data points are positive/1.0 and the next 12500 are
# negative/0.0 then the next 12500 is poitive and the fourth chunk is negative.
# So basically the train_data has 25K (with the first half positive and the second half negative)
# and test_data with the same setup for class label. 
# The rest of the data in the file is unknown and we don't use that part.
# We could write code to extract label automatically and we will do this based on a standardized format we will work with
# later, for now we will hard-code the labels.

from random import shuffle, randint


train_tags=[ 1.0 for i in range(12500)] + [ 0.0 for i in range(12500)]
test_tags=[ 1.0 for i in range(12500)] + [ 0.0 for i in range(12500)]


#train_tags=[ 1.0 for i in range(12500)] + [ 0.0 for i in range(12500)]
#test_tags=[ 1.0 for i in range(12500)] + [ 0.0 for i in range(12500)]
# Side note: If the first token in each line were the tag, we could get tags as follows:
# tags= [train_data[i].tag for i in range(len(train_data))]
print train_tags[-1], train_vecs[-1][:10]
print len(train_tags)
print len(test_tags)

0.0 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
25000
25000


In [5]:
train_vecs=np.array(train_vecs)
train_tags=np.array(train_tags)
print train_vecs.shape

(25000, 113562)


In [ ]:
# Classification with scikit-learn
# Now we have: train_tags, train_vecs, test_tags, test_vecs
# Let's use sklearn to train an svm classifier:
#-------------------------------------------------

import argparse
import codecs
import time
import sys
import os, re, glob
import nltk
from collections import defaultdict
from random import shuffle, randint
import numpy as np
from numpy import array, arange, zeros, hstack, argsort
import unicodedata
from scipy.sparse import csr_matrix
from sklearn.svm import SVC, LinearSVC
from sklearn import preprocessing
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
import gensim
n_jobs = 4

#train_vecs=array(train_vecs)
train_vecs=np.array(train_vecs)
train_tags=np.array(train_tags)

print type(train_tags)
print type(train_vecs)
clf = OneVsRestClassifier(SVC(C=1, kernel = 'linear', gamma=1, verbose= False, probability=False))
clf.fit(train_vecs, train_tags)
print "\nDone fitting classifier on training data...\n"

#------------------------------------------------------------------------------------------
print "="*50, "\n"
print "Results with 5-fold cross validation:\n"
print "="*50, "\n"
#------------------------------------------------------------------------------------------
predicted = cross_validation.cross_val_predict(clf, train_vecs, train_tags, cv=5)
print "*"*20
print "\t accuracy_score\t", metrics.accuracy_score(train_tags, predicted)
print "*"*20
print "precision_score\t", metrics.precision_score(train_tags, predicted)
print "recall_score\t", metrics.recall_score(train_tags, predicted)
print "\nclassification_report:\n\n", metrics.classification_report(train_tags, predicted)
print "\nconfusion_matrix:\n\n", metrics.confusion_matrix(train_tags, predicted)

In [ ]:
X= train_vecs
y=train_tags
y=y.astype(int)
num_examples = len(X) # training set size
nn_input_dim = len(train_vecs[0]) # input layer dimensionality
nn_output_dim = 2 # output layer dimensionality

# Gradient descent parameters (I picked these by hand)
epsilon = 0.01 # learning rate for gradient descent
reg_lambda = 0.01 # regularization strength 


def forward(W1, b1, W2, b2, x):
    z1 = x.dot(W1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    # softmax
    y_hat = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return y_hat, z1, a1, z2

def predict(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    y_hat, _, _, _ = forward(W1, b1, W2, b2, x)
    return np.argmax(y_hat, axis=1)

def calculate_loss(model):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    y_hat, _, _, _ = forward(W1, b1, W2, b2, X)
    correct_logprobs = -np.log(y_hat[range(num_examples), y])
    data_loss = np.sum(correct_logprobs)
    return 1./num_examples * data_loss


# This function learns parameters for the neural network and returns the model.
# - nn_hdim: Number of nodes in the hidden layer
# - num_passes: Number of passes through the training data for gradient descent
# - print_loss: If True, print the loss every 1000 iterations
def build_model(nn_hdim, num_passes=2000, print_loss=False):
    
    # Initialize the parameters to random values. We need to learn these.
    np.random.seed(0)
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((1, nn_hdim))
    W2 = np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)
    b2 = np.zeros((1, nn_output_dim))

    # This is what we return at the end
    model = {}
    
    # Gradient descent. For each batch...
    for i in range(0, num_passes):
        # feedforward
        y_hat, z1, a1, z2 = forward(W1, b1, W2, b2, X)
        
        # Backpropagation
        delta3 = y_hat
        delta3[range(num_examples), y] -= 1
        #print [range(num_examples), y]
        dW2 = (a1.T).dot(delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(W2.T) * (1 - np.power(a1, 2))
        dW1 = np.dot(X.T, delta2)
        db1 = np.sum(delta2, axis=0)

        # Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        
        # Optionally print the loss.
        # This is expensive because it uses the whole dataset, so we don't want to do it too often.
        if print_loss and i % 1000 == 0:
          print "Loss after iteration %i: %f" %(i, calculate_loss(model))
          #print y_hat[:2]
            
    
    return model

# Build a model with a 3-dimensional hidden layer
model = build_model(3, print_loss=True)


